In [1]:
from amplpy import AMPL, ampl_notebook

ampl = ampl_notebook(
    modules=["cbc", "highs"],  # modules to install
    license_uuid="default",  # license to use
)  # instantiate AMPL object and register magics

Licensed to Bundle #6850.7319 expiring 20241231: OR 7310 Logistics, Warehousing, and Scheduling, Prof. Ergun, Northeastern University.


In [2]:
# Optionally, you can clear any previous definitions if needed
ampl.reset()

In [ ]:
#Shortest Distance Path

import heapq

# Define the graph as an adjacency list
graph = {
    'Yellowstone': [('Livingston', 59), ('Idaho_Falls', 100)],
    'Livingston': [('Yellowstone', 59), ('Butte', 104)],
    'Idaho_Falls': [('Yellowstone', 100), ('Butte', 205), ('Salt_Lake_City', 208), ('Twin_Falls', 161)],
    'Butte': [('Livingston', 104), ('Idaho_Falls', 205), ('Missoula', 119)],
    'Missoula': [('Butte', 119), ('Boise', 371)],
    'Boise': [('Missoula', 371), ('Twin_Falls', 131), ('Winnemuca', 256)],
    'Salt_Lake_City': [('Idaho_Falls', 208), ('Wells', 181), ('Ely', 241)],
    'Twin_Falls': [('Idaho_Falls', 161), ('Boise', 131), ('Wells', 118)],
    'Wells': [('Twin_Falls', 118), ('Salt_Lake_City', 181), ('Winnemuca', 175), ('Ely', 140)],
    'Winnemuca': [('Boise', 256), ('Wells', 175), ('Reno', 164)],
    'Reno': [('Winnemuca', 164), ('Bishop', 205), ('Sacramento', 133), ('Yosemite', 154)],
    'Sacramento': [('Reno', 133), ('Yosemite', 193)],
    'Ely': [('Salt_Lake_City', 241), ('Wells', 140), ('Bishop', 283)],
    'Bishop': [('Ely', 283), ('Reno', 205), ('Yosemite', 65)],
    'Yosemite': [('Reno', 154), ('Sacramento', 193), ('Bishop', 65)]
}

# Dijkstra's algorithm to find the shortest path
def dijkstra(graph, start, end):
    # Priority queue to store (distance, node) and start from the source node
    pq = [(0, start)]  # (distance to node, node)
    distances = {node: float('inf') for node in graph}  # Initialize distances to infinity
    distances[start] = 0  # Start node has distance 0
    previous_nodes = {node: None for node in graph}  # Track the shortest path tree

    while pq:
        current_distance, current_node = heapq.heappop(pq)

        # If we reached the destination, stop
        if current_node == end:
            break

        # Explore neighbors of the current node
        for neighbor, weight in graph[current_node]:
            distance = current_distance + weight

            # If found a shorter path to the neighbor
            if distance < distances[neighbor]:
                distances[neighbor] = distance
                previous_nodes[neighbor] = current_node
                heapq.heappush(pq, (distance, neighbor))

    # Reconstruct the shortest path
    path = []
    node = end
    while previous_nodes[node] is not None:
        path.insert(0, node)
        node = previous_nodes[node]
    path.insert(0, start)

    return path, distances[end]

# Run Dijkstra's algorithm from Yellowstone to Yosemite
shortest_path, total_distance = dijkstra(graph, 'Yellowstone', 'Yosemite')

# Output the results
print(f"Shortest path: {' -> '.join(shortest_path)}")
print(f"Total distance: {total_distance} miles")


Shortest path: Yellowstone -> Idaho_Falls -> Twin_Falls -> Wells -> Ely -> Bishop -> Yosemite
Total distance: 867 miles


In [ ]:
#Shortest Time Path
import heapq

# Define the graph using the provided data
graph = {
    "Yellowstone_NP_WY": [("Livingston_MT", 84), ("Idaho_Falls_ID", 128)],
    "Livingston_MT": [("Butte_MT", 100)],
    "Butte_MT": [("Idaho_Falls_ID", 210), ("Missoula_MT", 110)],
    "Missoula_MT": [("Boise_ID", 475)],
    "Idaho_Falls_ID": [("Salt_Lake_City_UT", 206), ("Twin_Falls_ID", 155)],
    "Boise_ID": [("Twin_Falls_ID", 128), ("Winnemuca_NV", 303)],
    "Twin_Falls_ID": [("Wells_NV", 141)],
    "Salt_Lake_City_UT": [("Wells_NV", 174), ("Ely_NV", 262)],
    "Wells_NV": [("Winnemuca_NV", 162), ("Ely_NV", 180)],
    "Winnemuca_NV": [("Reno_NV", 153)],
    "Ely_NV": [("Bishop_CA", 337)],
    "Reno_NV": [("Bishop_CA", 255), ("Sacramento_CA", 152), ("Yosemite_NP_CA", 225)],
    "Sacramento_CA": [("Yosemite_NP_CA", 277)],
    "Bishop_CA": [("Yosemite_NP_CA", 132)],
    "Yosemite_NP_CA": []  # Destination node
}

def dijkstra(graph, start, end):
    # Create a priority queue to store (cost, node)
    queue = [(0, start)]
    # Create a dictionary to store the minimum cost to reach each node
    min_costs = {node: float('inf') for node in graph}
    min_costs[start] = 0
    # Create a dictionary to track the best path
    paths = {node: [] for node in graph}
    paths[start] = [start]

    while queue:
        current_cost, current_node = heapq.heappop(queue)

        # If we reach the end node, return the cost and the path
        if current_node == end:
            return current_cost, paths[current_node]

        # Explore neighbors
        for neighbor, travel_time in graph[current_node]:
            cost = current_cost + travel_time
            # If this path is cheaper, update the cost and the path
            if cost < min_costs[neighbor]:
                min_costs[neighbor] = cost
                paths[neighbor] = paths[current_node] + [neighbor]
                heapq.heappush(queue, (cost, neighbor))

    return float('inf'), []

# Define the source and destination
source = "Yellowstone_NP_WY"
destination = "Yosemite_NP_CA"

# Call Dijkstra's algorithm
shortest_time, path = dijkstra(graph, source, destination)

# Print the results
print(f"Shortest travel time from {source} to {destination}: {shortest_time} minutes")
print(f"Path taken: {' -> '.join(path)}")



Shortest travel time from Yellowstone_NP_WY to Yosemite_NP_CA: 964 minutes
Path taken: Yellowstone_NP_WY -> Idaho_Falls_ID -> Twin_Falls_ID -> Wells_NV -> Winnemuca_NV -> Reno_NV -> Yosemite_NP_CA


In [ ]:
#Multilabel_path
import heapq

# Define the graph with both distance and time for each edge
graph = {
    "Yellowstone_NP_WY": [("Livingston_MT", 84, 90), ("Idaho_Falls_ID", 128, 120)],
    "Livingston_MT": [("Butte_MT", 100, 120)],
    "Butte_MT": [("Idaho_Falls_ID", 210, 180), ("Missoula_MT", 110, 90)],
    "Missoula_MT": [("Boise_ID", 475, 360)],
    "Idaho_Falls_ID": [("Salt_Lake_City_UT", 206, 180), ("Twin_Falls_ID", 155, 140)],
    "Boise_ID": [("Twin_Falls_ID", 128, 120), ("Winnemuca_NV", 303, 240)],
    "Twin_Falls_ID": [("Wells_NV", 141, 130)],
    "Salt_Lake_City_UT": [("Wells_NV", 174, 160), ("Ely_NV", 262, 240)],
    "Wells_NV": [("Winnemuca_NV", 162, 140), ("Ely_NV", 180, 160)],
    "Winnemuca_NV": [("Reno_NV", 153, 130)],
    "Ely_NV": [("Bishop_CA", 337, 300)],
    "Reno_NV": [("Bishop_CA", 255, 220), ("Sacramento_CA", 152, 140), ("Yosemite_NP_CA", 225, 210)],
    "Sacramento_CA": [("Yosemite_NP_CA", 277, 240)],
    "Bishop_CA": [("Yosemite_NP_CA", 132, 110)],
    "Yosemite_NP_CA": []  # Destination node
}

# Define the time and distance constraints
MAX_TIME = 18 * 60  # 18 hours in minutes
MAX_DISTANCE = 900  # 900 miles

# Function to check if a new label is non-dominated
def is_non_dominated(new_label, labels):
    new_dist, new_time = new_label
    for (dist, time) in labels:
        if dist <= new_dist and time <= new_time:
            return False
    return True

# Multilabel Dijkstra algorithm
def multilabel_dijkstra(graph, start, end, max_distance=None, max_time=None):
    # Priority queue to store (distance, time, node, path)
    queue = [(0, 0, start, [start])]
    
    # Dictionary to store non-dominated labels for each node
    labels = {node: [] for node in graph}
    labels[start] = [(0, 0)]
    
    # Dictionary to store the paths corresponding to the labels
    paths = {start: [[start]]}
    
    while queue:
        current_dist, current_time, current_node, current_path = heapq.heappop(queue)
        
        # Stop if we reach the destination
        if current_node == end:
            print(f"Non-dominated path found: Distance = {current_dist} miles, Time = {current_time} minutes")
            print(f"Path: {' -> '.join(current_path)}")
            continue
        
        # Explore the neighbors
        for neighbor, distance, time in graph[current_node]:
            new_dist = current_dist + distance
            new_time = current_time + time
            
            # Apply constraints on distance and time
            if (max_distance is None or new_dist <= max_distance) and (max_time is None or new_time <= max_time):
                new_label = (new_dist, new_time)
                
                # Check if the new label is non-dominated
                if is_non_dominated(new_label, labels[neighbor]):
                    # Update the labels and add to the queue
                    labels[neighbor].append(new_label)
                    heapq.heappush(queue, (new_dist, new_time, neighbor, current_path + [neighbor]))
                    
                    # Keep track of the path
                    if neighbor not in paths:
                        paths[neighbor] = [current_path + [neighbor]]
                    else:
                        paths[neighbor].append(current_path + [neighbor])
    
    return labels, paths

# Call the multilabel Dijkstra function
source = "Yellowstone_NP_WY"
destination = "Yosemite_NP_CA"

# Solve for shortest distance subject to time constraint (18 hours)
print("### Shortest Distance with Time Constraint ###")
labels_distance, paths_distance = multilabel_dijkstra(graph, source, destination, max_time=MAX_TIME)

# Solve for shortest time subject to distance constraint (900 miles)
print("\n### Shortest Time with Distance Constraint ###")
labels_time, paths_time = multilabel_dijkstra(graph, source, destination, max_distance=MAX_DISTANCE)


### Shortest Distance with Time Constraint ###
Non-dominated path found: Distance = 964 miles, Time = 870 minutes
Path: Yellowstone_NP_WY -> Idaho_Falls_ID -> Twin_Falls_ID -> Wells_NV -> Winnemuca_NV -> Reno_NV -> Yosemite_NP_CA

### Shortest Time with Distance Constraint ###
